# 📊 Real Data Analysis: Option Pricing Model Comparison

This notebook calibrates and compares four stochastic volatility models using **real SPY option chain data**:

| Model | Description |
|-------|-------------|
| **Heston** | Stochastic volatility with mean-reversion |
| **Merton** | Jump-diffusion model (constant volatility + jumps) |
| **Bates** | Heston + Merton (stochastic vol + price jumps) |
| **SVJJ** | Bates + volatility jumps (most complex) |

---

## 1. Data Loading & Preprocessing

We download real-time SPY option chain data from Yahoo Finance and prepare it for calibration.

In [ ]:
# =============================================================================
# [MUST BE FIRST] Fix OpenMP Duplicate Library Error
# =============================================================================
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# Now import libraries
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import torch
import sys
import gc

# Add parent directory to path for imports
sys.path.append(os.path.abspath('..'))
from src.physics_engine import MarketSimulator

print("Libraries loaded successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

Libraries loaded successfully!
PyTorch version: 2.9.1+cu128
CUDA available: True


In [2]:
# =============================================================================
# Download SPY Option Chain Data
# =============================================================================
ticker = "SPY"
print(f"[{ticker}] Downloading option chain data...")

spy = yf.Ticker(ticker)

# Get current price
try:
    current_price = spy.history(period="1d")['Close'].iloc[-1]
    print(f"✅ Current Price (S0): ${current_price:.2f}")
except Exception as e:
    current_price = 580.0
    print(f"⚠️ Failed to load price. Using fallback: ${current_price}")

# Select expiration date (30-75 days out for good liquidity)
expirations = spy.options
target_date = None
today = datetime.now()

for exp_date in expirations:
    exp_dt = datetime.strptime(exp_date, "%Y-%m-%d")
    days_to_expire = (exp_dt - today).days
    if 30 <= days_to_expire <= 75:
        target_date = exp_date
        print(f"✅ Selected Expiration: {target_date} ({days_to_expire} days)")
        break

if target_date is None:
    target_date = expirations[min(3, len(expirations)-1)]
    exp_dt = datetime.strptime(target_date, "%Y-%m-%d")
    days_to_expire = (exp_dt - today).days
    print(f"⚠️ Fallback Expiration: {target_date} ({days_to_expire} days)")

[SPY] Downloading option chain data...
✅ Current Price (S0): $690.31
✅ Selected Expiration: 2026-01-30 (32 days)


In [3]:
# =============================================================================
# Data Cleaning & Preprocessing
# =============================================================================
opt_chain = spy.option_chain(target_date)
calls = opt_chain.calls

print(f"Raw call options: {len(calls)}")

# Filter for liquidity
calls_clean = calls[(calls['volume'] > 5) | (calls['openInterest'] > 10)].copy()

# Extract relevant columns
market_data = calls_clean[['strike', 'impliedVolatility', 'lastPrice']].copy()
market_data = market_data.sort_values('strike')

# Remove outliers
market_data = market_data[
    (market_data['impliedVolatility'] > 0.01) & 
    (market_data['impliedVolatility'] < 1.0)
]

# Focus on near-the-money options (80% - 120% of spot)
market_data = market_data[
    (market_data['strike'] > current_price * 0.8) & 
    (market_data['strike'] < current_price * 1.2)
]

print(f"✅ Preprocessed: {len(market_data)} valid data points")
market_data.head(10)

Raw call options: 215
✅ Preprocessed: 170 valid data points


,strike,impliedVolatility,lastPrice
23,555.0,0.528935,139.00
24,560.0,0.513372,119.44
25,565.0,0.498113,117.34
26,570.0,0.482549,113.54
28,573.0,0.473394,105.34
29,574.0,0.470281,115.50
30,575.0,0.467229,117.33
33,578.0,0.458074,100.31
35,580.0,0.451910,112.26
39,585.0,0.436834,108.88


In [4]:
# =============================================================================
# Define Calibration Variables
# =============================================================================
calib_strikes = market_data['strike'].values
calib_ivs = market_data['impliedVolatility'].values

# Time to Maturity (in years)
expiry_date = datetime.strptime(target_date, "%Y-%m-%d")
T_val = max((expiry_date - today).days / 365.0, 0.01)

# Risk-free Rate & Time Step
r_val = 0.04  # ~4% (current Fed rate)
dt_val = 1/252  # Daily time step

print("=" * 50)
print("Calibration Parameters")
print("=" * 50)
print(f"  S0 (Spot Price):    ${current_price:.2f}")
print(f"  T (Time to Expiry): {T_val:.4f} years ({int(T_val*365)} days)")
print(f"  r (Risk-free Rate): {r_val:.2%}")
print(f"  dt (Time Step):     {dt_val:.6f}")
print(f"  # Strikes:          {len(calib_strikes)}")
print("=" * 50)

Calibration Parameters
  S0 (Spot Price):    $690.31
  T (Time to Expiry): 0.0877 years (32 days)
  r (Risk-free Rate): 4.00%
  dt (Time Step):     0.003968
  # Strikes:          170


---

## 2. Helper Functions

We define the **Black-Scholes pricing formula** and an **implied volatility solver** using Brent's method.

In [5]:
from scipy.stats import norm
from scipy.optimize import brentq

def black_scholes_call_price(S, K, T, r, sigma):
    """
    Calculate Black-Scholes call option price.
    
    Args:
        S: Spot price
        K: Strike price
        T: Time to maturity (years)
        r: Risk-free rate
        sigma: Volatility
    
    Returns:
        Call option price
    """
    if sigma <= 0 or T <= 0:
        return 0.0
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)


def implied_vol_solver(market_price, S, K, T, r, sigma_low=0.001, sigma_high=5.0):
    """
    Solve for implied volatility using Brent's method.
    
    Args:
        market_price: Observed option price
        S, K, T, r: Standard BS parameters
        sigma_low/high: Search bounds for volatility
    
    Returns:
        Implied volatility (or np.nan if solver fails)
    """
    if market_price <= 0 or T <= 0 or S <= 0 or K <= 0:
        return np.nan
    
    # Check arbitrage bounds
    intrinsic = max(S - K * np.exp(-r * T), 0)
    if market_price < intrinsic or market_price >= S:
        return np.nan
    
    def objective(sigma):
        return black_scholes_call_price(S, K, T, r, sigma) - market_price
    
    try:
        f_low = objective(sigma_low)
        f_high = objective(sigma_high)
        
        if f_low * f_high > 0:
            return np.nan
        
        return brentq(objective, sigma_low, sigma_high, maxiter=100)
    except:
        return np.nan

print("✅ Helper functions defined!")

✅ Helper functions defined!


---

## 3. Calibration Loss Function

The loss function:
1. Unpacks model parameters
2. Runs Monte Carlo simulation using `MarketSimulator`
3. Calculates model-implied volatilities
4. Returns RMSE between model IVs and market IVs

In [6]:
def calibration_loss(params, model_name, market_strikes, market_ivs, S0, T, r, dt, num_paths):
    """
    Calculate RMSE between model-implied volatilities and market IVs.
    
    Supports: 'heston', 'merton', 'bates', 'svjj'
    """
    try:
        # =====================================================
        # Parameter Unpacking
        # =====================================================
        if model_name == 'heston':
            kappa, theta, xi, rho = params
            jump_params = {'jump_lambda': 0.0, 'jump_mean': 0.0, 'jump_std': 0.0}
            val_type = 'heston'
            
        elif model_name == 'merton':
            sigma, jump_lambda, jump_mean, jump_std = params
            kappa, theta, xi, rho = 1.0, sigma**2, 0.001, 0.0
            jump_params = {'jump_lambda': jump_lambda, 'jump_mean': jump_mean, 'jump_std': jump_std}
            val_type = 'bates'
            
        elif model_name == 'bates':
            kappa, theta, xi, rho, jump_lambda, jump_mean, jump_std = params
            jump_params = {'jump_lambda': jump_lambda, 'jump_mean': jump_mean, 'jump_std': jump_std}
            val_type = 'bates'
            
        elif model_name == 'svjj':
            kappa, theta, xi, rho, jump_lambda, jump_mean, jump_std, vol_jump_mean = params
            jump_params = {
                'jump_lambda': jump_lambda, 
                'jump_mean': jump_mean, 
                'jump_std': jump_std, 
                'vol_jump_mean': vol_jump_mean
            }
            val_type = 'svjj'
            if vol_jump_mean < 0:
                return 1e9
        else:
            return 1e9

        # Safety Checks
        if kappa < 0 or theta < 0 or xi < 0 or abs(rho) > 0.99:
            return 1e9
        if jump_params['jump_lambda'] < 0 or jump_params['jump_std'] < 0:
            return 1e9
            
    except Exception as e:
        return 1e9

    # =====================================================
    # Monte Carlo Simulation (Pure NumPy - No PyTorch)
    # =====================================================
    try:
        np.random.seed(42)  # Reproducibility
        n_steps = int(T / dt)
        
        # Initialize arrays
        S = np.zeros((num_paths, n_steps + 1))
        v = np.zeros((num_paths, n_steps + 1))
        S[:, 0] = S0
        v[:, 0] = theta
        
        sqrt_dt = np.sqrt(dt)
        
        # Drift correction for jump models
        drift_correction = 0.0
        if val_type in ['bates', 'svjj']:
            k = np.exp(jump_params['jump_mean'] + 0.5 * jump_params['jump_std']**2) - 1
            drift_correction = jump_params['jump_lambda'] * k * dt
        
        for t in range(1, n_steps + 1):
            z1 = np.random.randn(num_paths)
            z2 = np.random.randn(num_paths)
            
            dw_s = z1
            dw_v = rho * z1 + np.sqrt(1 - rho**2) * z2
            
            v_prev = np.maximum(v[:, t-1], 1e-8)
            S_prev = S[:, t-1]
            
            # Variance process
            dv = kappa * (theta - v_prev) * dt + xi * np.sqrt(v_prev) * sqrt_dt * dw_v
            
            # Asset process
            dS = (0.0 - drift_correction) * S_prev + np.sqrt(v_prev) * S_prev * sqrt_dt * dw_s
            
            # Jump process
            if val_type in ['bates', 'svjj']:
                jump_occur = np.random.rand(num_paths) < (jump_params['jump_lambda'] * dt)
                if np.any(jump_occur):
                    log_jump = np.random.normal(jump_params['jump_mean'], jump_params['jump_std'], num_paths)
                    jump_S = (np.exp(log_jump) - 1) * S_prev * jump_occur
                    dS += jump_S
                    
                    if val_type == 'svjj':
                        vol_jump = np.random.exponential(jump_params['vol_jump_mean'], num_paths)
                        dv += vol_jump * jump_occur
            
            v[:, t] = np.maximum(v_prev + dv, 1e-8)
            S[:, t] = S_prev + dS
        
        S_final = S[:, -1]
        
        if np.isnan(S_final).any() or np.mean(S_final) < 1e-3:
            return 1e9
        
        # Martingale Correction
        S_corr = S_final * (S0 / np.mean(S_final))
        
        # =====================================================
        # Calculate RMSE
        # =====================================================
        error_sum = 0.0
        valid_count = 0
        
        for i, K in enumerate(market_strikes):
            payoff = np.maximum(S_corr - K, 0)
            model_price = np.mean(payoff) * np.exp(-r * T)
            m_iv = implied_vol_solver(model_price, S0, K, T, r)
            target_iv = market_ivs[i]
            
            if np.isnan(m_iv) or m_iv < 0.001 or m_iv > 5.0:
                error_sum += 0.1
            else:
                error_sum += (m_iv - target_iv) ** 2
                valid_count += 1
        
        if valid_count < len(market_strikes) * 0.3:
            return 1e9
                
        return np.sqrt(error_sum / len(market_strikes))
        
    except Exception as e:
        return 1e9

print("✅ Calibration loss function defined (Pure NumPy version)!")

✅ Calibration loss function defined (Pure NumPy version)!


---

## 4. Model Calibration

We use **Differential Evolution** (a global optimizer) to find the best parameters for each model.

In [ ]:
from scipy.optimize import differential_evolution
import gc

# Clear memory before starting
gc.collect()

# Optimization settings (minimal for stability)
base_opts = {
    'strategy': 'best1bin',
    'maxiter': 8,
    'popsize': 5,
    'tol': 0.05,
    'mutation': (0.5, 1),
    'recombination': 0.7,
    'workers': 1,
    'updating': 'deferred',
    'seed': 42
}

# Very small paths for stability
N_paths = 300
calib_args = (calib_strikes, calib_ivs, current_price, T_val, r_val, dt_val, N_paths)

print("=" * 60)
print(f"Starting Model Calibration (N={N_paths} paths)")
print("=" * 60)

Starting Model Calibration (N=300 paths)


: 

In [ ]:
# =============================================================================
# 1. Heston Model
# =============================================================================
print("\n[1/4] 🔄 Calibrating Heston Model...")
print("     Parameters: kappa, theta, xi, rho")

bounds_heston = [
    (0.5, 3.0),   # kappa: mean-reversion speed (narrower)
    (0.02, 0.1),  # theta: long-term variance (narrower)
    (0.2, 1.0),   # xi: vol-of-vol (narrower)
    (-0.8, -0.2)  # rho: correlation (typically negative)
]

try:
    res_heston = differential_evolution(
        calibration_loss, bounds_heston, 
        args=('heston', *calib_args), 
        **base_opts
    )
    print(f"     ✅ Heston RMSE: {res_heston.fun:.4f}")
    print(f"     Parameters: kappa={res_heston.x[0]:.3f}, theta={res_heston.x[1]:.4f}, xi={res_heston.x[2]:.3f}, rho={res_heston.x[3]:.3f}")
except Exception as e:
    print(f"     ❌ Heston Error: {e}")
    res_heston = type('obj', (object,), {'x': [2.0, 0.04, 0.5, -0.7], 'fun': 1e9})()

gc.collect()


[1/4] 🔄 Calibrating Heston Model...
     Parameters: kappa, theta, xi, rho


In [ ]:
# =============================================================================
# 2. Merton Model
# =============================================================================
print("\n[2/4] 🔄 Calibrating Merton Model...")
print("     Parameters: sigma, jump_lambda, jump_mean, jump_std")

bounds_merton = [
    (0.1, 0.4),   # sigma
    (0.5, 3.0),   # jump_lambda
    (-0.2, 0.0),  # jump_mean
    (0.05, 0.2)   # jump_std
]

try:
    res_merton = differential_evolution(
        calibration_loss, bounds_merton, 
        args=('merton', *calib_args), 
        **base_opts
    )
    print(f"     ✅ Merton RMSE: {res_merton.fun:.4f}")
    print(f"     Parameters: sigma={res_merton.x[0]:.3f}, lambda={res_merton.x[1]:.3f}, mu_j={res_merton.x[2]:.3f}, sigma_j={res_merton.x[3]:.3f}")
except Exception as e:
    print(f"     ❌ Merton Error: {e}")
    res_merton = type('obj', (object,), {'x': [0.2, 1.0, -0.1, 0.1], 'fun': 1e9})()

gc.collect()

In [ ]:
# =============================================================================
# 3. Bates Model
# =============================================================================
print("\n[3/4] 🔄 Calibrating Bates Model...")
print("     Parameters: kappa, theta, xi, rho, jump_lambda, jump_mean, jump_std")

bounds_bates = [
    (0.5, 3.0),    # kappa
    (0.02, 0.1),   # theta
    (0.2, 1.0),    # xi
    (-0.8, -0.3),  # rho
    (0.1, 3.0),    # jump_lambda
    (-0.2, 0.0),   # jump_mean
    (0.05, 0.2)    # jump_std
]

try:
    res_bates = differential_evolution(
        calibration_loss, bounds_bates, 
        args=('bates', *calib_args), 
        **base_opts
    )
    print(f"     ✅ Bates RMSE: {res_bates.fun:.4f}")
except Exception as e:
    print(f"     ❌ Bates Error: {e}")
    res_bates = type('obj', (object,), {'x': [2.0, 0.04, 0.5, -0.7, 0.5, -0.1, 0.1], 'fun': 1e9})()

gc.collect()

In [ ]:
# =============================================================================
# 4. SVJJ Model
# =============================================================================
print("\n[4/4] 🔄 Calibrating SVJJ Model...")
print("     Parameters: kappa, theta, xi, rho, jump_lambda, jump_mean, jump_std, vol_jump_mean")

bounds_svjj = bounds_bates + [(0.01, 0.1)]  # Add vol_jump_mean

try:
    res_svjj = differential_evolution(
        calibration_loss, bounds_svjj, 
        args=('svjj', *calib_args), 
        **base_opts
    )
    print(f"     ✅ SVJJ RMSE: {res_svjj.fun:.4f}")
except Exception as e:
    print(f"     ❌ SVJJ Error: {e}")
    res_svjj = type('obj', (object,), {'x': [2.0, 0.04, 0.5, -0.7, 0.5, -0.1, 0.1, 0.05], 'fun': 1e9})()

gc.collect()

print("\n" + "=" * 60)
print("Calibration Complete!")
print("=" * 60)

---

## 5. Visualization

We generate implied volatility curves for each calibrated model and compare them to market data.

In [ ]:
def generate_model_ivs(params, model_name, strikes, S0, T, r, dt, N):
    """
    Generate implied volatility curve for a calibrated model.
    """
    try:
        if model_name == 'heston':
            kappa, theta, xi, rho = params
            jump_params = {'jump_lambda': 0.0, 'jump_mean': 0.0, 'jump_std': 0.0}
            val_type = 'heston'
        elif model_name == 'merton':
            sigma, jump_lambda, jump_mean, jump_std = params
            kappa, theta, xi, rho = 1.0, sigma**2, 0.001, 0.0
            jump_params = {'jump_lambda': jump_lambda, 'jump_mean': jump_mean, 'jump_std': jump_std}
            val_type = 'bates'
        elif model_name == 'bates':
            kappa, theta, xi, rho, jump_lambda, jump_mean, jump_std = params
            jump_params = {'jump_lambda': jump_lambda, 'jump_mean': jump_mean, 'jump_std': jump_std}
            val_type = 'bates'
        elif model_name == 'svjj':
            kappa, theta, xi, rho, jump_lambda, jump_mean, jump_std, vol_jump_mean = params
            jump_params = {'jump_lambda': jump_lambda, 'jump_mean': jump_mean, 'jump_std': jump_std, 'vol_jump_mean': vol_jump_mean}
            val_type = 'svjj'

        # Use GPU for visualization if available
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        with torch.no_grad():
            sim = MarketSimulator(mu=0.0, kappa=kappa, theta=theta, xi=xi, rho=rho, device=device, **jump_params)
            S_paths, _ = sim.simulate(S0, theta, T, dt, N, model_type=val_type)
            
            S_final = S_paths[:, -1]
            mean_S = S_final.mean()
            
            if mean_S > 1e-3 and not torch.isnan(mean_S):
                S_corr = S_final * (S0 / mean_S)
            else:
                S_corr = S_final
            
            S_corr_np = S_corr.cpu().numpy()
            del S_paths, S_final, S_corr, sim
            
            if device == 'cuda':
                torch.cuda.empty_cache()
        
        gc.collect()

        ivs = []
        for K in strikes:
            payoff = np.maximum(S_corr_np - K, 0)
            model_price = np.mean(payoff) * np.exp(-r * T)
            try:
                iv = implied_vol_solver(model_price, S0, K, T, r)
            except:
                iv = np.nan
            ivs.append(iv)
        
        return np.array(ivs)
    
    except Exception as e:
        print(f"Generation Error ({model_name}): {e}")
        return np.full(len(strikes), np.nan)

print("✅ Visualization function defined!")

In [ ]:
# Generate IV curves for plotting
print("Generating IV curves for visualization...")

N_plot = 3000
strikes_plot = np.linspace(market_data['strike'].min(), market_data['strike'].max(), 40)

print("  → Heston...", end=" ")
iv_heston = generate_model_ivs(res_heston.x, 'heston', strikes_plot, current_price, T_val, r_val, dt_val, N_plot)
print("Done!")
gc.collect()

print("  → Merton...", end=" ")
iv_merton = generate_model_ivs(res_merton.x, 'merton', strikes_plot, current_price, T_val, r_val, dt_val, N_plot)
print("Done!")
gc.collect()

print("  → Bates...", end=" ")
iv_bates = generate_model_ivs(res_bates.x, 'bates', strikes_plot, current_price, T_val, r_val, dt_val, N_plot)
print("Done!")
gc.collect()

print("  → SVJJ...", end=" ")
iv_svjj = generate_model_ivs(res_svjj.x, 'svjj', strikes_plot, current_price, T_val, r_val, dt_val, N_plot)
print("Done!")
gc.collect()

In [ ]:
# =============================================================================
# Plot Results
# =============================================================================
plt.figure(figsize=(14, 8))

# Market data
plt.scatter(market_data['strike'], market_data['impliedVolatility'], 
            c='black', s=50, alpha=0.6, label='Market Data (SPY)', zorder=5)

# Model curves
plt.plot(strikes_plot, iv_heston, 'g:', linewidth=2.5, 
         label=f'Heston (RMSE={res_heston.fun:.4f})')
plt.plot(strikes_plot, iv_merton, 'c-.', linewidth=2.5, 
         label=f'Merton (RMSE={res_merton.fun:.4f})')
plt.plot(strikes_plot, iv_bates, 'b-', linewidth=3, alpha=0.8, 
         label=f'Bates (RMSE={res_bates.fun:.4f})')
plt.plot(strikes_plot, iv_svjj, 'r--', linewidth=3, 
         label=f'SVJJ (RMSE={res_svjj.fun:.4f})')

# Spot price line
plt.axvline(current_price, color='gray', linestyle=':', alpha=0.7, linewidth=2, label='Spot Price')

# Labels and styling
plt.xlabel('Strike Price ($)', fontsize=14)
plt.ylabel('Implied Volatility', fontsize=14)
plt.title('The Evolution of Option Pricing Models: Real Data Calibration', fontsize=16, fontweight='bold')
plt.legend(loc='upper right', fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

---

## 6. Final Leaderboard

Ranking models by calibration accuracy (lower RMSE = better fit to market data).

In [ ]:
# =============================================================================
# Final Leaderboard
# =============================================================================
results = {
    'Heston': res_heston.fun,
    'Merton': res_merton.fun,
    'Bates': res_bates.fun,
    'SVJJ': res_svjj.fun
}

sorted_res = sorted(results.items(), key=lambda x: x[1])

print("\n" + "=" * 50)
print("        🏆 FINAL LEADERBOARD 🏆")
print("=" * 50)

medals = ["🥇", "🥈", "🥉", "  "]
for rank, (name, score) in enumerate(sorted_res):
    print(f"  {medals[rank]} {rank+1}. {name:<10} | RMSE: {score:.5f}")

print("=" * 50)
print(f"\n🎯 Best Model: {sorted_res[0][0]} with RMSE = {sorted_res[0][1]:.5f}")

---

## Summary

This analysis demonstrates the progression of option pricing models:

1. **Heston**: Captures volatility smile but may miss extreme tails
2. **Merton**: Jump component helps with fat tails but lacks stochastic vol flexibility
3. **Bates**: Combines both, typically performs well
4. **SVJJ**: Most flexible, can capture complex volatility surface dynamics

The "best" model depends on the specific market conditions and the trade-off between accuracy and model complexity.